## Question 1: 

You suspect that certain hoteliers are committing fraud by canceling bookings from
agencies and then manually adding them again to avoid paying commissions. 
Your **task** is to identify such fraudulent activities using only SQL. 
View enclosed CSV file bookings.csv.

bookings Table:

● booking_id: Unique identifier for the booking
● hotel_id: Identifier for the hotel
● country: the property location country
● booking_date: Date the booking was made
● cancellation_date: Date the booking was canceled (if applicable)
● checkin_date: Date of check-in
● checkout_date: Date of check-out
● status: Status of the booking (e.g., 'confirmed', 'canceled')
● source: Source of the booking (e.g., 'agency', 'manual')
● agency_id: Identifier for the agency (if applicable)

For each agency_id, identify patterns where bookings are canceled and then a new manual booking is
made for the same hotel, and check-in date within a short time frame (e.g., within one day).

What other metrics or analyses would you perform to verify if the identified fraudulent activities are out
of the norm?

## Approach of the first task:

We downloaded the data in the first place, to import it in MySql Workbench.
The big size of the file bookings.csv made some members of the team have trouble when trying to import it, so the first team member who succeded in doing so, passed the .sql file to the other team members.



    ``` sql
    WITH cancelaciones AS (
     SELECT
        agency_id,
        hotel_id,
        checkin_date,
        booking_date AS fecha_cancelacion
     FROM bookings
     WHERE status = 'cancelled'
    ),
    nuevas_reservas AS (
     SELECT
        agency_id,
        hotel_id,
        checkin_date,
        booking_date AS fecha_nueva,
        source
     FROM bookings
     WHERE source = 'booking.com' OR source = 'manual'
    )
    SELECT
        c.agency_id,
        c.hotel_id,
        c.checkin_date,
        c.fecha_cancelacion,
        n.fecha_nueva
    FROM cancelaciones c
    JOIN nuevas_reservas n
        ON c.agency_id = n.agency_id
        AND c.hotel_id = n.hotel_id
        AND c.checkin_date = n.checkin_date
        AND DATEDIFF(n.fecha_nueva, c.fecha_cancelacion) BETWEEN 0 AND 1
    ORDER BY c.agency_id, c.hotel_id, c.checkin_date, c.fecha_cancelacion;
    
    ```

This analysis focuses on identifying possible fraudulent activities in which reservations made by agencies are canceled and, in a short period of time (within a day), new reservations are created manually or through booking.com for the same hotel and date check-in. 
The logic used is based on **dividing reservations into two groups**: cancellations (cancellations) and new reservations (nuevas_reservas). 
Both lists are then crossed using a **JOIN to find matches** based on specific criteria, such as the same hotel_id, checkin_date, and a short interval between cancellation and rebooking.


The results show 19,873 records, all associated with agency_id number 2. 
This suggests that this specific agency has **a recurring pattern** in which its reservations are quickly canceled and replaced by manual or booking.com reservations. 

Such a volume of matches could indicate that related hotels are trying to avoid paying commissions to the agency by reusing the same dates and room assignments. 
However, there may also be other valid explanations (such as specific agency policies or technical failures), so this analysis must be complemented with additional metrics to confirm the fraud hypothesis.
